<a href="https://colab.research.google.com/github/parmidamardi/CDM_TA_5/blob/main/CDM_TA_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Section 1

In [ ]:
#step 1.1

In [7]:
import pandas as pd

X = pd.read_csv(
    "https://raw.githubusercontent.com/parmidamardi/CDM_TA_5/refs/heads/main/secom.data",
    sep=" ",
    header=None
)

y = pd.read_csv(
    "https://raw.githubusercontent.com/parmidamardi/CDM_TA_5/refs/heads/main/secom_labels.data",
    header=None
)

In [8]:
print(X.shape)
print(y.shape)


(1567, 590)
(1567, 1)


In [9]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=0.0)
X_no_const = vt.fit_transform(X)


In [10]:
print("Original features:", X.shape[1])
print("Remaining features:", X_no_const.shape[1])
print("Removed zero-variance features:", X.shape[1] - X_no_const.shape[1])


Original features: 590
Remaining features: 474
Removed zero-variance features: 116


In [11]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X_no_const)


In [12]:
print("Remaining NaNs:", np.isnan(X_imputed).sum())


Remaining NaNs: 0


In [13]:
#step 1.2

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)


In [15]:
import numpy as np

print(np.mean(X_scaled, axis=0)[:5])
print(np.std(X_scaled, axis=0)[:5])

[ 1.62332035e-15 -5.07372631e-15 -5.71336214e-15 -1.26963603e-16
  0.00000000e+00]
[1. 1. 1. 1. 1.]
